In [ ]:
import numpy as np
import pickle
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, r2_score
from math import sqrt
from tabulate import tabulate

pd.options.display.max_columns = None

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
with open("/content/drive/MyDrive/8 seconds/train_df_ECG_8s_corr.pkl", "rb") as file_name:
  data_train_8s = pickle.load(file_name)

with open("/content/drive/MyDrive/8 seconds/test_df_ECG_8s_corr.pkl", "rb") as file_name:
  data_test_8s = pickle.load(file_name)

with open("/content/drive/MyDrive/15 seconds/train_df_ECG_15s.pkl", "rb") as file_name:
  data_train_15s = pickle.load(file_name)

with open("/content/drive/MyDrive/15 seconds/test_df_ECG_15s.pkl", "rb") as file_name:
  data_test_15s = pickle.load(file_name)

with open("/content/drive/MyDrive/30 seconds/train_df_ECG_30s.pkl", "rb") as file_name:
  data_train_30s = pickle.load(file_name)

with open("/content/drive/MyDrive/30 seconds/test_df_ECG_30s.pkl", "rb") as file_name:
  data_test_30s = pickle.load(file_name)

In [ ]:
#Ovoj del go imam ovde staveno bidejki koga gi sreduvav podatocite imav problem so e-sdsd kolonata, no od kolegata vidov kako da se spravam so nejze
data_train_8s['e-sdsd'] = data_train_8s['e-sdsd'].astype('float64')
data_train_15s['e-sdsd'] = data_train_15s['e-sdsd'].astype('float64')
data_train_30s['e-sdsd'] = data_train_30s['e-sdsd'].astype('float64')

/usr/local/lib/python3.7/dist-packages/pandas/core/dtypes/cast.py:1201: UserWarning: Warning: converting a masked element to nan.
  return arr.astype(dtype, copy=True)


In [ ]:
print(data_train_8s.shape)
print(data_test_8s.shape)
print(data_train_15s.shape)
print(data_test_15s.shape)
print(data_train_30s.shape)
print(data_test_30s.shape)

(275154, 104)
(83381, 104)
(208145, 104)
(31570, 104)
(100660, 104)
(19256, 104)


In [ ]:
data_train_8s.head()

,R-wave,S-wave,Q-wave,P-wave,T-wave,RR-median,RR-mode,RR-mean,RR-min,RR-max,RR-std,RR-skew,RR-kurtosis,PR-median,PR-mode,PR-mean,PR-min,PR-max,PR-std,PR-skew,PR-kurtosis,QRS-median,QRS-mode,QRS-mean,QRS-min,QRS-max,QRS-std,QRS-skew,QRS-kurtosis,QT-median,QT-mode,QT-mean,QT-min,QT-max,QT-std,QT-skew,QT-kurtosis,QTC-median,QTC-mode,QTC-mean,QTC-min,QTC-max,QTC-std,QTC-skew,QTC-kurtosis,BPM-median,BPM-mode,BPM-mean,BPM-min,BPM-max,BPM-std,BPM-skew,BPM-kurtosis,e-btb,e-hr,HRV_MeanNN,HRV_SDNN,HRV_RMSSD,HRV_SDSD,HRV_CVNN,HRV_CVSD,HRV_MedianNN,HRV_MadNN,HRV_MCVNN,HRV_IQRNN,HRV_Prc20NN,HRV_Prc80NN,HRV_pNN50,HRV_pNN20,HRV_MinNN,HRV_MaxNN,HRV_HTI,HRV_TINN,e-activity,e-morbidity,e-complexity,e-ShanEn,e-SampEn,e-FuzzyEn,e-PSD,e-CD,e-HFD,e-KFD,e-bpm,e-ibi,e-sdnn,e-sdsd,e-rmssd,e-pnn20,e-pnn50,e-hr_mad,e-sd1,e-sd2,e-s,e-sd1/sd2,e-breathingrate,SBP,DBP,FILE,ID,ECG_type,ECG_quality,Hz,MAP
0,0.561765,-0.127706,-0.032310,0.077908,0.008694,0.928,0.928,0.928000,0.920,0.936,0.004000,0.000000,1.000000,0.184,0.184,0.184000,0.176,0.192,0.005657,0.000000,-1.000000,0.216,0.216,0.222857,0.184,0.264,0.025091,0.076545,-0.983239,0.476961,0.240973,0.423965,0.240973,0.520965,0.106988,-0.944290,-0.854378,0.507579,0.249076,0.466218,0.249076,0.540797,0.093818,-1.624556,1.205123,64.655172,64.655172,64.656545,64.102564,65.217391,0.297958,0.034557,0.500425,0.928,64.641949,928.000000,4.276180,7.406561,8.000000,0.004608,0.007981,928.0,0.0,0.0,0.0,928.0,928.0,0.0,0.0,920.0,936.0,1.333333,0.0,0.012727,0.368856,1.927496,9.965784,0.138050,0.206267,3.073580,0.513326,1.891068,1.829077,64.655172,928.000000,4.000000,5.827451,7.406561,0.0,0.0,0.0,5.237229,3.023716,49.749925,1.732051,0.134698,83.555556,49.700000,mimic2_0.pkl,3000003_0007,II,Barely acceptable,125.0,60.985185
1,0.563697,-0.132785,-0.037771,0.079977,0.017445,0.920,0.920,0.924571,0.920,0.936,0.005827,0.859894,-0.636095,0.184,0.184,0.184000,0.168,0.192,0.007407,-1.080123,0.500000,0.232,0.232,0.226286,0.192,0.248,0.019926,-0.770882,-1.047827,1.400957,1.400957,1.205533,0.216965,1.456973,0.410575,-1.906470,1.847009,1.435577,0.225225,1.219660,0.225225,1.505961,0.452444,-1.661812,0.950296,64.936282,65.217391,64.844180,64.102564,65.217391,0.416193,-0.613662,-0.982229,0.920,64.893167,924.571429,6.294366,8.640988,9.352362,0.006808,0.009346,920.0,0.0,0.0,8.0,920.0,928.0,0.0,0.0,920.0,936.0,1.750000,0.0,0.011007,0.381332,1.873708,9.965784,0.148276,0.215940,2.880079,0.515939,1.891595,1.828379,64.894932,924.571429,5.827451,5.497474,8.640988,0.0,0.0,0.0,6.036923,6.036923,114.493599,1.000000,0.154512,86.488889,50.666667,mimic2_0.pkl,3000003_0007,II,Barely acceptable,125.0,62.607407
2,0.574391,-0.130847,-0.030382,0.076539,0.006391,0.928,0.928,0.926857,0.920,0.936,0.005111,0.134164,-0.610000,0.184,0.184,0.181714,0.168,0.192,0.007045,-0.665962,-0.231302,0.232,0.200,0.224000,0.200,0.248,0.017631,-0.240217,-1.413495,0.488965,0.184957,0.510678,0.184957,1.344965,0.362589,1.550248,1.155906,0.403772,0.192831,0.385723,0.192831,0.530221,0.138673,-0.145932,-1.775957,64.655172,64.655172,64.750477,64.102564,65.217391,0.383636,-0.209589,-0.910764,0.928,64.743139,926.857143,5.520524,5.656854,6.022181,0.005956,0.006103,928.0,0.0,0.0,4.0,921.6,928.0,0.0,0.0,920.0,936.0,1.750000,0.0,0.012021,0.383739,1.867562,9.965784,0.123766,0.205696,2.943427,0.531966,1.903421,1.879993,64.794816,926.000000,5.291503,3.958973,5.237229,0.0,0.0,4.0,3.614032,6.663945,75.661200,0.542326,0.134989,86.400000,50.400000,mimic2_0.pkl,3000003_0007,II,Barely acceptable,125.0,62.400000
3,0.574336,-0.133792,-0.031987,0.073671,0.017512,0.936,0.936,0.937143,0.928,0.944,0.005111,-0.134164,-0.610000,0.184,0.184,0.187429,0.184,0.200,0.005827,1.357727,0.233728,0.224,0.224,0.219429,0.192,0.232,0.011986,-1.518606,1.102975,0.464982,0.464973,0.472975,0.408965,0.528973,0.033673,-0.299301,-0.026147,0.491942,0.424534,0.489861,0.424534,0.546759,0.036833,-0.302331,-0.391307,64.102564,64.102564,64.013585,63.559322,64.655172,0.375965,0.246930,-0.874538,0.936,64.014888,937.142857,5.520524,9.237

In [ ]:
data_train_8s.isna().sum()

R-wave            0
S-wave            0
Q-wave           22
P-wave          412
T-wave         2465
               ... 
ID                0
ECG_type          0
ECG_quality       0
Hz                0
MAP               0
Length: 104, dtype: int64

Uspeav da gi najdam kolonite so NaN vrednosti, jas ke trgnev racno da gi brisam site koloni koi so smetav deka imaat premnogu NaN vrednosti, no potoa vidov od kolegata kako bi mozelo toa da se avtomatizira, pa se nadevam deka ne e problem sto go iskoristiv negoviot kod za da odredam tocno koi koloni od datasetot treba da se izbrisat. 

In [ ]:
to_drop = list()
max_missing = data_train_8s.shape[0] * 0.01
for col in data_train_8s.columns:
  if data_train_8s[col].isna().sum() > max_missing:
    to_drop.append(col)

to_drop

['QT-median',
 'QT-mode',
 'QT-mean',
 'QT-min',
 'QT-max',
 'QT-std',
 'QT-skew',
 'QT-kurtosis',
 'QTC-median',
 'QTC-mode',
 'QTC-mean',
 'QTC-min',
 'QTC-max',
 'QTC-std',
 'QTC-skew',
 'QTC-kurtosis',
 'e-bpm',
 'e-ibi',
 'e-sdnn',
 'e-sdsd',
 'e-rmssd',
 'e-pnn20',
 'e-pnn50',
 'e-hr_mad',
 'e-sd1',
 'e-sd2',
 'e-s',
 'e-sd1/sd2',
 'e-breathingrate']

Ostanatite NaN vrednosti gi srediv taka sto gi izbrisav redovite vo koi sto se pojavuvaat, bidejki vo sporedba so goleminata na datasetot nemase mnogu.

In [ ]:
data_train_8s = data_train_8s.drop(columns=to_drop)
data_test_8s = data_test_8s.drop(columns=to_drop)
data_train_8s = data_train_8s.dropna()
data_test_8s = data_test_8s.dropna()


In [ ]:
print(data_train_8s.shape)
print(data_test_8s.shape)

(271919, 75)
(63447, 75)


In [ ]:
to_drop = list()
max_missing = data_train_15s.shape[0] * 0.01
for col in data_train_15s.columns:
  if data_train_15s[col].isna().sum() > max_missing:
    to_drop.append(col)

to_drop

['T-wave',
 'QT-median',
 'QT-mode',
 'QT-mean',
 'QT-min',
 'QT-max',
 'QT-std',
 'QT-skew',
 'QT-kurtosis',
 'QTC-median',
 'QTC-mode',
 'QTC-mean',
 'QTC-min',
 'QTC-max',
 'QTC-std',
 'QTC-skew',
 'QTC-kurtosis',
 'e-bpm',
 'e-ibi',
 'e-sdnn',
 'e-sdsd',
 'e-rmssd',
 'e-pnn20',
 'e-pnn50',
 'e-hr_mad',
 'e-sd1',
 'e-sd2',
 'e-s',
 'e-sd1/sd2',
 'e-breathingrate']

In [ ]:
data_train_15s = data_train_15s.drop(columns=to_drop)
data_test_15s = data_test_15s.drop(columns=to_drop)
data_train_15s = data_train_15s.dropna()
data_test_15s = data_test_15s.dropna()

In [ ]:
print(data_train_15s.shape)
print(data_test_15s.shape)

(206732, 74)
(31538, 74)


In [ ]:
to_drop = list()
max_missing = data_train_30s.shape[0] * 0.01
for col in data_train_30s.columns:
  if data_train_30s[col].isna().sum() > max_missing:
    to_drop.append(col)

to_drop

['e-sdsd',
 'e-rmssd',
 'e-pnn20',
 'e-pnn50',
 'e-sd1',
 'e-sd2',
 'e-s',
 'e-sd1/sd2',
 'e-breathingrate']

In [ ]:
data_train_30s = data_train_30s.drop(columns=to_drop)
data_test_30s = data_test_30s.drop(columns=to_drop)
data_train_30s = data_train_30s.dropna()
data_test_30s = data_test_30s.dropna()

In [ ]:
print(data_train_30s.shape)
print(data_test_30s.shape)

(99535, 95)
(12183, 95)


In [ ]:
X_train_8s, y_train_8s = data_train_8s.drop(['FILE', 'ID', 'ECG_type', 'ECG_quality', 'Hz', 'SBP', 'DBP', 'MAP'], axis=1), data_train_8s[['SBP', 'DBP', 'MAP']]
X_train_15s, y_train_15s = data_train_15s.drop(['FILE', 'ID', 'ECG_type', 'ECG_quality', 'Hz', 'SBP', 'DBP', 'MAP'], axis=1), data_train_15s[['SBP', 'DBP', 'MAP']]
X_train_30s, y_train_30s = data_train_30s.drop(['FILE', 'ID', 'ECG_type', 'ECG_quality', 'Hz', 'SBP', 'DBP', 'MAP'], axis=1), data_train_30s[['SBP', 'DBP', 'MAP']]
X_test_8s, y_test_8s = data_test_8s.drop(['FILE', 'ID', 'ECG_type', 'ECG_quality', 'Hz', 'SBP', 'DBP', 'MAP'], axis=1), data_test_8s[['SBP', 'DBP', 'MAP']]
X_test_15s, y_test_15s = data_test_15s.drop(['FILE', 'ID', 'ECG_type', 'ECG_quality', 'Hz', 'SBP', 'DBP', 'MAP'], axis=1), data_test_15s[['SBP', 'DBP', 'MAP']]
X_test_30s, y_test_30s = data_test_30s.drop(['FILE', 'ID', 'ECG_type', 'ECG_quality', 'Hz', 'SBP', 'DBP', 'MAP'], axis=1), data_test_30s[['SBP', 'DBP', 'MAP']]

In [ ]:
X_train_8s.head()

,R-wave,S-wave,Q-wave,P-wave,T-wave,RR-median,RR-mode,RR-mean,RR-min,RR-max,RR-std,RR-skew,RR-kurtosis,PR-median,PR-mode,PR-mean,PR-min,PR-max,PR-std,PR-skew,PR-kurtosis,QRS-median,QRS-mode,QRS-mean,QRS-min,QRS-max,QRS-std,QRS-skew,QRS-kurtosis,BPM-median,BPM-mode,BPM-mean,BPM-min,BPM-max,BPM-std,BPM-skew,BPM-kurtosis,e-btb,e-hr,HRV_MeanNN,HRV_SDNN,HRV_RMSSD,HRV_SDSD,HRV_CVNN,HRV_CVSD,HRV_MedianNN,HRV_MadNN,HRV_MCVNN,HRV_IQRNN,HRV_Prc20NN,HRV_Prc80NN,HRV_pNN50,HRV_pNN20,HRV_MinNN,HRV_MaxNN,HRV_HTI,HRV_TINN,e-activity,e-morbidity,e-complexity,e-ShanEn,e-SampEn,e-FuzzyEn,e-PSD,e-CD,e-HFD,e-KFD
0,0.561765,-0.127706,-0.032310,0.077908,0.008694,0.928,0.928,0.928000,0.920,0.936,0.004000,0.000000,1.000000,0.184,0.184,0.184000,0.176,0.192,0.005657,0.000000,-1.000000,0.216,0.216,0.222857,0.184,0.264,0.025091,0.076545,-0.983239,64.655172,64.655172,64.656545,64.102564,65.217391,0.297958,0.034557,0.500425,0.928,64.641949,928.000000,4.276180,7.406561,8.000000,0.004608,0.007981,928.0,0.0,0.0,0.0,928.0,928.0,0.0,0.0,920.0,936.0,1.333333,0.0,0.012727,0.368856,1.927496,9.965784,0.138050,0.206267,3.073580,0.513326,1.891068,1.829077
1,0.563697,-0.132785,-0.037771,0.079977,0.017445,0.920,0.920,0.924571,0.920,0.936,0.005827,0.859894,-0.636095,0.184,0.184,0.184000,0.168,0.192,0.007407,-1.080123,0.500000,0.232,0.232,0.226286,0.192,0.248,0.019926,-0.770882,-1.047827,64.936282,65.217391,64.844180,64.102564,65.217391,0.416193,-0.613662,-0.982229,0.920,64.893167,924.571429,6.294366,8.640988,9.352362,0.006808,0.009346,920.0,0.0,0.0,8.0,920.0,928.0,0.0,0.0,920.0,936.0,1.750000,0.0,0.011007,0.381332,1.873708,9.965784,0.148276,0.215940,2.880079,0.515939,1.891595,1.828379
2,0.574391,-0.130847,-0.030382,0.076539,0.006391,0.928,0.928,0.926857,0.920,0.936,0.005111,0.134164,-0.610000,0.184,0.184,0.181714,0.168,0.192,0.007045,-0.665962,-0.231302,0.232,0.200,0.224000,0.200,0.248,0.017631,-0.240217,-1.413495,64.655172,64.655172,64.750477,64.102564,65.217391,0.383636,-0.209589,-0.910764,0.928,64.743139,926.857143,5.520524,5.656854,6.022181,0.005956,0.006103,928.0,0.0,0.0,4.0,921.6,928.0,0.0,0.0,920.0,936.0,1.750000,0.0,0.012021,0.383739,1.867562,9.965784,0.123766,0.205696,2.943427,0.531966,1.903421,1.879993
3,0.574336,-0.133792,-0.031987,0.073671,0.017512,0.936,0.936,0.937143,0.928,0.944,0.005111,-0.134164,-0.610000,0.184,0.184,0.187429,0.184,0.200,0.005827,1.357727,0.233728,0.224,0.224,0.219429,0.192,0.232,0.011986,-1.518606,1.102975,64.102564,64.102564,64.013585,63.559322,64.655172,0.375965,0.246930,-0.874538,0.936,64.014888,937.142857,5.520524,9.237604,10.119289,0.005891,0.009857,936.0,0.0,0.0,4.0,936.0,942.4,0.0,0.0,928.0,944.0,1.750000,0.0,0.012495,0.380872,1.845764,9.965784,0.126652,0.205955,2.902667,0.511059,1.898165,1.871322
4,0.580525,-0.138851,-0.039863,0.072244,0.012561,0.928,0.928,0.932571,0.928,0.944,0.005827,0.859894,-0.636095,0.184,0.192,0.185143,0.176,0.192,0.006664,-0.272380,-1.506920,0.224,0.224,0.220571,0.184,0.272,0.027666,0.466954,-0.654457,64.378868,64.655172,64.288328,63.559322,64.655172,0.409103,-0.613768,-0.982040,0.928,64.356744,932.571429,6.294366,5.656854,6.022181,0.006749,0.006066,928.0,0.0,0.0,8.0,928.0,936.0,0.0,0.0,928.0,944.0,1.750000,0.0,0.013040,0.364131,1.968860,9.965784,0.138918,0.202312,3.236475,0.462914,1.879002,1.861879


In [ ]:
y_train_8s.head()

,SBP,DBP,MAP
0,83.555556,49.700000,60.985185
1,86.488889,50.666667,62.607407
2,86.400000,50.400000,62.400000
3,89.955556,50.933333,63.940741
4,85.777778,50.300000,62.125926


In [ ]:
scaler = MinMaxScaler()
X_train_8s = pd.DataFrame(scaler.fit_transform(X_train_8s.values), columns=X_train_8s.columns)
X_test_8s = pd.DataFrame(scaler.transform(X_test_8s.values), columns=X_test_8s.columns)
X_train_15s = pd.DataFrame(scaler.fit_transform(X_train_15s.values), columns=X_train_15s.columns)
X_test_15s = pd.DataFrame(scaler.transform(X_test_15s.values), columns=X_test_15s.columns)
X_train_30s = pd.DataFrame(scaler.fit_transform(X_train_30s.values), columns=X_train_30s.columns)
X_test_30s = pd.DataFrame(scaler.transform(X_test_30s.values), columns=X_test_30s.columns)

In [ ]:
sbp_model = LinearRegression()
sbp_model.fit(X_train_8s, y_train_8s['SBP'])
sbp_pred = sbp_model.predict(X_test_8s)

dbp_model = LinearRegression()
dbp_model.fit(X_train_8s, y_train_8s['DBP'])
dbp_pred = dbp_model.predict(X_test_8s)

map_model = LinearRegression()
map_model.fit(X_train_8s, y_train_8s['MAP'])
map_pred = map_model.predict(X_test_8s)

lin_res = [[mean_absolute_error(y_test_8s['SBP'], sbp_pred), r2_score(y_test_8s['SBP'], sbp_pred), 
            mean_squared_error(y_test_8s['SBP'], sbp_pred), sqrt(mean_squared_error(y_test_8s['SBP'], sbp_pred)), 
            mean_absolute_percentage_error(y_test_8s['SBP'], sbp_pred)], [mean_absolute_error(y_test_8s['DBP'], dbp_pred), r2_score(y_test_8s['DBP'], dbp_pred), 
            mean_squared_error(y_test_8s['DBP'], dbp_pred), sqrt(mean_squared_error(y_test_8s['DBP'], dbp_pred)), 
            mean_absolute_percentage_error(y_test_8s['DBP'], dbp_pred)], [mean_absolute_error(y_test_8s['MAP'], map_pred), r2_score(y_test_8s['MAP'], map_pred), 
            mean_squared_error(y_test_8s['MAP'], map_pred), sqrt(mean_squared_error(y_test_8s['MAP'], map_pred)), 
            mean_absolute_percentage_error(y_test_8s['MAP'], map_pred)]]

print('Results for Linear Regression for 8s data:')
print('MAE:')
print(mean_absolute_error(y_test_8s['SBP'], sbp_pred))
print(mean_absolute_error(y_test_8s['DBP'], dbp_pred))
print(mean_absolute_error(y_test_8s['MAP'], map_pred))

print('r2 score:')
print(r2_score(y_test_8s['SBP'], sbp_pred))
print(r2_score(y_test_8s['DBP'], dbp_pred))
print(r2_score(y_test_8s['MAP'], map_pred))

print('MSE:')
print(mean_squared_error(y_test_8s['SBP'], sbp_pred))
print(mean_squared_error(y_test_8s['DBP'], dbp_pred))
print(mean_squared_error(y_test_8s['MAP'], map_pred))

print('RMSE:')
print(sqrt(mean_squared_error(y_test_8s['SBP'], sbp_pred)))
print(sqrt(mean_squared_error(y_test_8s['DBP'], dbp_pred)))
print(sqrt(mean_squared_error(y_test_8s['MAP'], map_pred)))

print('MAPE:')
print(mean_absolute_percentage_error(y_test_8s['SBP'], sbp_pred))
print(mean_absolute_percentage_error(y_test_8s['DBP'], dbp_pred))
print(mean_absolute_percentage_error(y_test_8s['MAP'], map_pred))

Results for Linear Regression for 8s data:
MAE:
18.573417986893148
12.000714092261665
13.364374764940528
r2 score:
-0.07669057330100548
-0.21910722961961482
-0.14772676246549254
MSE:
657.7651310033157
327.2179062056167
381.2613893967994
RMSE:
25.646932194773623
18.08916543695747
19.52591584015458
MAPE:
0.1511629216290824
0.19839152786311598
0.1637495948838638


In [ ]:
sbp_model = LinearRegression()
sbp_model.fit(X_train_15s, y_train_15s['SBP'])
sbp_pred = sbp_model.predict(X_test_15s)

dbp_model = LinearRegression()
dbp_model.fit(X_train_15s, y_train_15s['DBP'])
dbp_pred = dbp_model.predict(X_test_15s)

map_model = LinearRegression()
map_model.fit(X_train_15s, y_train_15s['MAP'])
map_pred = map_model.predict(X_test_15s)

lin_res.append([mean_absolute_error(y_test_15s['SBP'], sbp_pred), r2_score(y_test_15s['SBP'], sbp_pred), 
            mean_squared_error(y_test_15s['SBP'], sbp_pred), sqrt(mean_squared_error(y_test_15s['SBP'], sbp_pred)), 
            mean_absolute_percentage_error(y_test_15s['SBP'], sbp_pred)])
lin_res.append([mean_absolute_error(y_test_15s['DBP'], dbp_pred), r2_score(y_test_15s['DBP'], dbp_pred), 
            mean_squared_error(y_test_15s['DBP'], dbp_pred), sqrt(mean_squared_error(y_test_15s['DBP'], dbp_pred)), 
            mean_absolute_percentage_error(y_test_15s['DBP'], dbp_pred)])
lin_res.append([mean_absolute_error(y_test_15s['MAP'], map_pred), r2_score(y_test_15s['MAP'], map_pred), 
            mean_squared_error(y_test_15s['MAP'], map_pred), sqrt(mean_squared_error(y_test_15s['MAP'], map_pred)), 
            mean_absolute_percentage_error(y_test_15s['MAP'], map_pred)])

print('Results for Linear Regression for 15s data:')
print('MAE:')
print(mean_absolute_error(y_test_15s['SBP'], sbp_pred))
print(mean_absolute_error(y_test_15s['DBP'], dbp_pred))
print(mean_absolute_error(y_test_15s['MAP'], map_pred))

print('r2 score:')
print(r2_score(y_test_15s['SBP'], sbp_pred))
print(r2_score(y_test_15s['DBP'], dbp_pred))
print(r2_score(y_test_15s['MAP'], map_pred))

print('MSE:')
print(mean_squared_error(y_test_15s['SBP'], sbp_pred))
print(mean_squared_error(y_test_15s['DBP'], dbp_pred))
print(mean_squared_error(y_test_15s['MAP'], map_pred))

print('RMSE:')
print(sqrt(mean_squared_error(y_test_15s['SBP'], sbp_pred)))
print(sqrt(mean_squared_error(y_test_15s['DBP'], dbp_pred)))
print(sqrt(mean_squared_error(y_test_15s['MAP'], map_pred)))

print('MAPE:')
print(mean_absolute_percentage_error(y_test_15s['SBP'], sbp_pred))
print(mean_absolute_percentage_error(y_test_15s['DBP'], dbp_pred))
print(mean_absolute_percentage_error(y_test_15s['MAP'], map_pred))

Results for Linear Regression for 15s data:
MAE:
14.771817819480708
7.624283742114129
8.962686901885785
r2 score:
-0.5670658175181522
-0.35390272315070015
-0.4096499391776609
MSE:
350.8572093451197
97.0464309659781
134.08084351890216
RMSE:
18.731182806889684
9.851214694948949
11.57932828444302
MAPE:
0.13702627486474123
0.13451815091834826
0.1209797173175121


In [ ]:
sbp_model = LinearRegression()
sbp_model.fit(X_train_30s, y_train_30s['SBP'])
sbp_pred = sbp_model.predict(X_test_30s)

dbp_model = LinearRegression()
dbp_model.fit(X_train_30s, y_train_30s['DBP'])
dbp_pred = dbp_model.predict(X_test_30s)

map_model = LinearRegression()
map_model.fit(X_train_30s, y_train_30s['MAP'])
map_pred = map_model.predict(X_test_30s)

lin_res.append([mean_absolute_error(y_test_30s['SBP'], sbp_pred), r2_score(y_test_30s['SBP'], sbp_pred), 
            mean_squared_error(y_test_30s['SBP'], sbp_pred), sqrt(mean_squared_error(y_test_30s['SBP'], sbp_pred)), 
            mean_absolute_percentage_error(y_test_30s['SBP'], sbp_pred)])
lin_res.append([mean_absolute_error(y_test_30s['DBP'], dbp_pred), r2_score(y_test_30s['DBP'], dbp_pred), 
            mean_squared_error(y_test_30s['DBP'], dbp_pred), sqrt(mean_squared_error(y_test_30s['DBP'], dbp_pred)), 
            mean_absolute_percentage_error(y_test_30s['DBP'], dbp_pred)])
lin_res.append([mean_absolute_error(y_test_30s['MAP'], map_pred), r2_score(y_test_30s['MAP'], map_pred), 
            mean_squared_error(y_test_30s['MAP'], map_pred), sqrt(mean_squared_error(y_test_30s['MAP'], map_pred)), 
            mean_absolute_percentage_error(y_test_30s['MAP'], map_pred)])

print('Results for Linear Regression for 30s data:')
print('MAE:')
print(mean_absolute_error(y_test_30s['SBP'], sbp_pred))
print(mean_absolute_error(y_test_30s['DBP'], dbp_pred))
print(mean_absolute_error(y_test_30s['MAP'], map_pred))

print('r2 score:')
print(r2_score(y_test_30s['SBP'], sbp_pred))
print(r2_score(y_test_30s['DBP'], dbp_pred))
print(r2_score(y_test_30s['MAP'], map_pred))

print('MSE:')
print(mean_squared_error(y_test_30s['SBP'], sbp_pred))
print(mean_squared_error(y_test_30s['DBP'], dbp_pred))
print(mean_squared_error(y_test_30s['MAP'], map_pred))

print('RMSE:')
print(sqrt(mean_squared_error(y_test_30s['SBP'], sbp_pred)))
print(sqrt(mean_squared_error(y_test_30s['DBP'], dbp_pred)))
print(sqrt(mean_squared_error(y_test_30s['MAP'], map_pred)))

print('MAPE:')
print(mean_absolute_percentage_error(y_test_30s['SBP'], sbp_pred))
print(mean_absolute_percentage_error(y_test_30s['DBP'], dbp_pred))
print(mean_absolute_percentage_error(y_test_30s['MAP'], map_pred))

Results for Linear Regression for 30s data:
MAE:
24.972969894100267
15.574191248489656
17.90328088878039
r2 score:
-0.6616240847127703
-0.1975010720864656
-0.3603478718192188
MSE:
1158.9846427769169
540.2600433603837
661.2647338468614
RMSE:
34.04386351131312
23.24349464603771
25.715068225592198
MAPE:
0.18859890563866832
0.2504795328327108
0.20583800066503316


In [ ]:
print(tabulate(lin_res, headers=['MAE', 'r2 score', 'MSE', 'RMSE', 'MAPE'], tablefmt='fancy_grid', showindex=['SBP_8s', 'DBP_8s', 'MAP_8s', 'SBP_15s', 'DBP_15s', 'MAP_15s', 'SBP_30s', 'DBP_30s', 'MAP_30s']))

╒═════════╤══════════╤════════════╤═══════════╤══════════╤══════════╕
│         │      MAE │   r2 score │       MSE │     RMSE │     MAPE │
╞═════════╪══════════╪════════════╪═══════════╪══════════╪══════════╡
│ SBP_8s  │ 18.5734  │ -0.0766906 │  657.765  │ 25.6469  │ 0.151163 │
├─────────┼──────────┼────────────┼───────────┼──────────┼──────────┤
│ DBP_8s  │ 12.0007  │ -0.219107  │  327.218  │ 18.0892  │ 0.198392 │
├─────────┼──────────┼────────────┼───────────┼──────────┼──────────┤
│ MAP_8s  │ 13.3644  │ -0.147727  │  381.261  │ 19.5259  │ 0.16375  │
├─────────┼──────────┼────────────┼───────────┼──────────┼──────────┤
│ SBP_15s │ 14.7718  │ -0.567066  │  350.857  │ 18.7312  │ 0.137026 │
├─────────┼──────────┼────────────┼───────────┼──────────┼──────────┤
│ DBP_15s │  7.62428 │ -0.353903  │   97.0464 │  9.85121 │ 0.134518 │
├─────────┼──────────┼────────────┼───────────┼──────────┼──────────┤
│ MAP_15s │  8.96269 │ -0.40965   │  134.081  │ 11.5793  │ 0.12098  │
├─────────┼─────────

In [ ]:
sbp_model = DecisionTreeRegressor()
sbp_model.fit(X_train_8s, y_train_8s['SBP'])
sbp_pred = sbp_model.predict(X_test_8s)

dbp_model = DecisionTreeRegressor()
dbp_model.fit(X_train_8s, y_train_8s['DBP'])
dbp_pred = dbp_model.predict(X_test_8s)

map_model = DecisionTreeRegressor()
map_model.fit(X_train_8s, y_train_8s['MAP'])
map_pred = map_model.predict(X_test_8s)

DT_res = [[mean_absolute_error(y_test_8s['SBP'], sbp_pred), r2_score(y_test_8s['SBP'], sbp_pred), 
            mean_squared_error(y_test_8s['SBP'], sbp_pred), sqrt(mean_squared_error(y_test_8s['SBP'], sbp_pred)), 
            mean_absolute_percentage_error(y_test_8s['SBP'], sbp_pred)], [mean_absolute_error(y_test_8s['DBP'], dbp_pred), r2_score(y_test_8s['DBP'], dbp_pred), 
            mean_squared_error(y_test_8s['DBP'], dbp_pred), sqrt(mean_squared_error(y_test_8s['DBP'], dbp_pred)), 
            mean_absolute_percentage_error(y_test_8s['DBP'], dbp_pred)], [mean_absolute_error(y_test_8s['MAP'], map_pred), r2_score(y_test_8s['MAP'], map_pred), 
            mean_squared_error(y_test_8s['MAP'], map_pred), sqrt(mean_squared_error(y_test_8s['MAP'], map_pred)), 
            mean_absolute_percentage_error(y_test_8s['MAP'], map_pred)]]

print('Results for Decision Tree Regression for 8s data:')
print('MAE:')
print(mean_absolute_error(y_test_8s['SBP'], sbp_pred))
print(mean_absolute_error(y_test_8s['DBP'], dbp_pred))
print(mean_absolute_error(y_test_8s['MAP'], map_pred))

print('r2 score:')
print(r2_score(y_test_8s['SBP'], sbp_pred))
print(r2_score(y_test_8s['DBP'], dbp_pred))
print(r2_score(y_test_8s['MAP'], map_pred))

print('MSE:')
print(mean_squared_error(y_test_8s['SBP'], sbp_pred))
print(mean_squared_error(y_test_8s['DBP'], dbp_pred))
print(mean_squared_error(y_test_8s['MAP'], map_pred))

print('RMSE:')
print(sqrt(mean_squared_error(y_test_8s['SBP'], sbp_pred)))
print(sqrt(mean_squared_error(y_test_8s['DBP'], dbp_pred)))
print(sqrt(mean_squared_error(y_test_8s['MAP'], map_pred)))

print('MAPE:')
print(mean_absolute_percentage_error(y_test_8s['SBP'], sbp_pred))
print(mean_absolute_percentage_error(y_test_8s['DBP'], dbp_pred))
print(mean_absolute_percentage_error(y_test_8s['MAP'], map_pred))

Results for Decision Tree Regression for 8s data:
MAE:
22.93124170982292
15.911446190714233
15.52436936039578
r2 score:
-0.4301972281525961
-0.7595581478217872
-0.32601336536705827
MSE:
873.7272253180345
472.27915558906676
440.4861109560217
RMSE:
29.55887726754916
21.73198462149895
20.98776098005744
MAPE:
0.1909608796409082
0.26888787842155065
0.19885172264388767


In [ ]:
sbp_model = DecisionTreeRegressor()
sbp_model.fit(X_train_15s, y_train_15s['SBP'])
sbp_pred = sbp_model.predict(X_test_15s)

dbp_model = DecisionTreeRegressor()
dbp_model.fit(X_train_15s, y_train_15s['DBP'])
dbp_pred = dbp_model.predict(X_test_15s)

map_model = DecisionTreeRegressor()
map_model.fit(X_train_15s, y_train_15s['MAP'])
map_pred = map_model.predict(X_test_15s)

DT_res.append([mean_absolute_error(y_test_15s['SBP'], sbp_pred), r2_score(y_test_15s['SBP'], sbp_pred), 
            mean_squared_error(y_test_15s['SBP'], sbp_pred), sqrt(mean_squared_error(y_test_15s['SBP'], sbp_pred)), 
            mean_absolute_percentage_error(y_test_15s['SBP'], sbp_pred)])
DT_res.append([mean_absolute_error(y_test_15s['DBP'], dbp_pred), r2_score(y_test_15s['DBP'], dbp_pred), 
            mean_squared_error(y_test_15s['DBP'], dbp_pred), sqrt(mean_squared_error(y_test_15s['DBP'], dbp_pred)), 
            mean_absolute_percentage_error(y_test_15s['DBP'], dbp_pred)])
DT_res.append([mean_absolute_error(y_test_15s['MAP'], map_pred), r2_score(y_test_15s['MAP'], map_pred), 
            mean_squared_error(y_test_15s['MAP'], map_pred), sqrt(mean_squared_error(y_test_15s['MAP'], map_pred)), 
            mean_absolute_percentage_error(y_test_15s['MAP'], map_pred)])

print('Results for Decision Tree Regression for 15s data:')
print('MAE:')
print(mean_absolute_error(y_test_15s['SBP'], sbp_pred))
print(mean_absolute_error(y_test_15s['DBP'], dbp_pred))
print(mean_absolute_error(y_test_15s['MAP'], map_pred))

print('r2 score:')
print(r2_score(y_test_15s['SBP'], sbp_pred))
print(r2_score(y_test_15s['DBP'], dbp_pred))
print(r2_score(y_test_15s['MAP'], map_pred))

print('MSE:')
print(mean_squared_error(y_test_15s['SBP'], sbp_pred))
print(mean_squared_error(y_test_15s['DBP'], dbp_pred))
print(mean_squared_error(y_test_15s['MAP'], map_pred))

print('RMSE:')
print(sqrt(mean_squared_error(y_test_15s['SBP'], sbp_pred)))
print(sqrt(mean_squared_error(y_test_15s['DBP'], dbp_pred)))
print(sqrt(mean_squared_error(y_test_15s['MAP'], map_pred)))

print('MAPE:')
print(mean_absolute_percentage_error(y_test_15s['SBP'], sbp_pred))
print(mean_absolute_percentage_error(y_test_15s['DBP'], dbp_pred))
print(mean_absolute_percentage_error(y_test_15s['MAP'], map_pred))

Results for Decision Tree Regression for 15s data:
MAE:
19.632545605639596
8.956415726552082
11.61222501643227
r2 score:
-1.6026040493300018
-0.887968197468765
-1.374060207133283
MSE:
582.7083863168084
135.32772496035574
225.8120873064303
RMSE:
24.13935347760599
11.633044526707346
15.027045195461092
MAPE:
0.17382968373685997
0.15438004343839332
0.15216189334325347


In [ ]:
sbp_model = DecisionTreeRegressor()
sbp_model.fit(X_train_30s, y_train_30s['SBP'])
sbp_pred = sbp_model.predict(X_test_30s)

dbp_model = DecisionTreeRegressor()
dbp_model.fit(X_train_30s, y_train_30s['DBP'])
dbp_pred = dbp_model.predict(X_test_30s)

map_model = DecisionTreeRegressor()
map_model.fit(X_train_30s, y_train_30s['MAP'])
map_pred = map_model.predict(X_test_30s)

DT_res.append([mean_absolute_error(y_test_30s['SBP'], sbp_pred), r2_score(y_test_30s['SBP'], sbp_pred), 
            mean_squared_error(y_test_30s['SBP'], sbp_pred), sqrt(mean_squared_error(y_test_30s['SBP'], sbp_pred)), 
            mean_absolute_percentage_error(y_test_30s['SBP'], sbp_pred)])
DT_res.append([mean_absolute_error(y_test_30s['DBP'], dbp_pred), r2_score(y_test_30s['DBP'], dbp_pred), 
            mean_squared_error(y_test_30s['DBP'], dbp_pred), sqrt(mean_squared_error(y_test_30s['DBP'], dbp_pred)), 
            mean_absolute_percentage_error(y_test_30s['DBP'], dbp_pred)])
DT_res.append([mean_absolute_error(y_test_30s['MAP'], map_pred), r2_score(y_test_30s['MAP'], map_pred), 
            mean_squared_error(y_test_30s['MAP'], map_pred), sqrt(mean_squared_error(y_test_30s['MAP'], map_pred)), 
            mean_absolute_percentage_error(y_test_30s['MAP'], map_pred)])

print('Results for Decision Tree Regression for 30s data:')
print('MAE:')
print(mean_absolute_error(y_test_30s['SBP'], sbp_pred))
print(mean_absolute_error(y_test_30s['DBP'], dbp_pred))
print(mean_absolute_error(y_test_30s['MAP'], map_pred))

print('r2 score:')
print(r2_score(y_test_30s['SBP'], sbp_pred))
print(r2_score(y_test_30s['DBP'], dbp_pred))
print(r2_score(y_test_30s['MAP'], map_pred))

print('MSE:')
print(mean_squared_error(y_test_30s['SBP'], sbp_pred))
print(mean_squared_error(y_test_30s['DBP'], dbp_pred))
print(mean_squared_error(y_test_30s['MAP'], map_pred))

print('RMSE:')
print(sqrt(mean_squared_error(y_test_30s['SBP'], sbp_pred)))
print(sqrt(mean_squared_error(y_test_30s['DBP'], dbp_pred)))
print(sqrt(mean_squared_error(y_test_30s['MAP'], map_pred)))

print('MAPE:')
print(mean_absolute_percentage_error(y_test_30s['SBP'], sbp_pred))
print(mean_absolute_percentage_error(y_test_30s['DBP'], dbp_pred))
print(mean_absolute_percentage_error(y_test_30s['MAP'], map_pred))

Results for Decision Tree Regression for 30s data:
MAE:
25.20155442510527
19.329786012373443
17.72910565094218
r2 score:
-0.681701350074434
-0.5916508783424923
-0.17140807122322244
MSE:
1172.9885576438269
718.0831755329054
569.4211477006891
RMSE:
34.248920532533965
26.79707401066216
23.86254696591898
MAPE:
0.1932567506892521
0.33409178032270287
0.21873443642491056


In [ ]:
print(tabulate(DT_res, headers=['MAE', 'r2 score', 'MSE', 'RMSE', 'MAPE'], tablefmt='fancy_grid', showindex=['SBP_8s', 'DBP_8s', 'MAP_8s', 'SBP_15s', 'DBP_15s', 'MAP_15s', 'SBP_30s', 'DBP_30s', 'MAP_30s']))

╒═════════╤══════════╤════════════╤══════════╤═════════╤══════════╕
│         │      MAE │   r2 score │      MSE │    RMSE │     MAPE │
╞═════════╪══════════╪════════════╪══════════╪═════════╪══════════╡
│ SBP_8s  │ 22.9312  │  -0.430197 │  873.727 │ 29.5589 │ 0.190961 │
├─────────┼──────────┼────────────┼──────────┼─────────┼──────────┤
│ DBP_8s  │ 15.9114  │  -0.759558 │  472.279 │ 21.732  │ 0.268888 │
├─────────┼──────────┼────────────┼──────────┼─────────┼──────────┤
│ MAP_8s  │ 15.5244  │  -0.326013 │  440.486 │ 20.9878 │ 0.198852 │
├─────────┼──────────┼────────────┼──────────┼─────────┼──────────┤
│ SBP_15s │ 19.6325  │  -1.6026   │  582.708 │ 24.1394 │ 0.17383  │
├─────────┼──────────┼────────────┼──────────┼─────────┼──────────┤
│ DBP_15s │  8.95642 │  -0.887968 │  135.328 │ 11.633  │ 0.15438  │
├─────────┼──────────┼────────────┼──────────┼─────────┼──────────┤
│ MAP_15s │ 11.6122  │  -1.37406  │  225.812 │ 15.027  │ 0.152162 │
├─────────┼──────────┼────────────┼──────────┼──